In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql import *
from pyspark.mllib.stat import Statistics
import pandas as pd
import pingouin as pg
import org.apache.spark.sql.{SaveMode,SparkSession}
spark = SparkSession.builder.appName('assignment 1').getOrCreate()

#read given csv file 
table2 = spark.read.csv('C:/Users/91738/Desktop/Assignment/nonConfidential.csv',header=True)
#table2.repartition(1).write.mode('overwrite').parquet('C:/Users/91738/Desktop/Assignment/nonConfidential.parquet')
#table2.show()


#read given parquet file and write to csv
df = spark.read.parquet('C:/Users/91738/Desktop/Assignment/confidential.snappy.parquet')
df.write.csv('C:/Users/91738/Desktop/Assignment/confidential_new.csv')
#df.show()

#merge both csv files
new_file=spark.read.format("csv").option("header", "true").load("C:/Users/91738/Desktop/Assignment/nonConfidential.csv","C:/Users/91738/Desktop/Assignment/C:/Users/91738/Desktop/Assignment/confidential_new.csv")
new_file.printSchema()

new_file.createOrReplaceTempView("new_table")


#using Spark SQL required query output is pushed into csv file in respective subdirectories of  /tmp/output directory
#Number of LEED projects there in Virginia
df1=spark.sql('select count("LEEDSystemVersionDisplayName") from new_table where state="VA"')
df1.write.save("/tmp/output/LEED_projects_VA/query1_result.csv")

#Number of LEED projects in Virginia by owner type
df2=spark.sql('select OwnerTypes,count("LEEDSystemVersionDisplayName") as count_proj from new_table where state="VA" groupby OwnerTypes')
df2.write.save("/tmp/output/Num_LEED_projects/query2_result.csv")


#Total Gross Square Feet of building space that is LEED-certified in Virginia
df3=spark.sql('select sum("GrossSqFoot") as total_gross_sq_feet from new_table where state="VA" and IsCertified="Yes"')
df3.write.save("/tmp/output/Gross_sqfeet_space/query3_result.csv")

#Zip Code in Virginia having high number of  projects
df4=spark.sql('select TOP 1 Zipcode COUNT (*) AS zip_count from new_table where state="VA" GROUP BY Zipcode ORDER BY zip_count DESC')
df4.write.save("/tmp/output/Num_Project_Zip_code/query4_result.csv")

#significant difference (use a t-test) 
#merging two csv file using pandas 
t_test_df=pd.read_csv("C:/Users/91738/Desktop/Assignment/nonConfidential.csv","C:/Users/91738/Desktop/Assignment/C:/Users/91738/Desktop/Assignment/confidential_new.csv")

virginia = t_test_df.query('grouping == "VA"')['PointsAchieved']
california = t_test_df('grouping == "CA"')['PointsAchieved']

res = pg.ttest(virginia, california, correction=False)
df5=res.summary(0)
df5.write.save("/tmp/output/Diff_CAnVI/query5_result.csv")